# Tensorflow examples

## 1. Logistic regression

In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import missingno as mso
%matplotlib inline

In [118]:
# Reset Tensorflow graphs
tf.reset_default_graph()

**Dataset**:

In [119]:
data = pd.read_csv('data.csv', index_col=0)
data.head()

#mso.matrix(X)
#plt.show()

X = data.drop('label', axis=1)
Y = data['label']

**Standardization of features**:

In [120]:
from sklearn.preprocessing import RobustScaler
X_std = RobustScaler().fit_transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)
X_std.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64
0,-0.345821,0.887681,-0.660175,-0.397176,-1.104933,0.000000,-0.334331,-0.400739,-0.319530,-0.841382,...,-0.441176,-0.183836,-0.344380,-0.082795,5.800039,-0.390605,0.059432,1.625507,-0.581995,-0.371594
1,0.706456,0.545661,0.074550,-0.113452,-1.212006,0.000000,0.557345,-0.292296,0.532584,-0.496989,...,-0.132985,1.762131,1.189227,-1.859282,1.333261,-0.515367,1.007042,0.689136,-0.386496,0.728880
2,1.999788,1.077786,-0.307964,-0.285190,0.074683,-6.970828,1.696353,-0.446301,2.078102,-1.032616,...,-0.211032,0.298691,11.103705,-0.351665,-0.031617,0.000000,6.636748,0.409183,-0.276270,46.279400
3,-1.566659,-0.978427,1.246802,6.710924,1.021185,8.135263,-1.444535,7.150731,0.465426,1.037126,...,-0.143239,-1.353773,-1.019099,1.066443,-0.031617,0.114174,0.860983,-0.802267,4.637867,0.353713
4,-0.103130,-0.089700,0.871831,0.369691,0.349257,0.000000,0.054180,0.102414,3.824296,0.142554,...,-0.128184,-0.256477,-0.201698,0.300112,-0.031617,0.000000,0.551877,-0.545997,1.215165,19.924162


**Train/Validation/Test split**

In [121]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X_std, Y.values, test_size=0.3)

**Build the model framework**

In [133]:
# Model data
n = X.shape[1]

# Hyperparameters
alpha = 0.01
num_epochs = 100
batch_size = 100
display_step = 10

# Tensorflow parameters
with tf.name_scope('Placeholder'):
    X = tf.placeholder(tf.float32, [None, n])
    Y = tf.placeholder(tf.float32, [None, 1])

with tf.name_scope('Variables'):
    W = tf.Variable(tf.zeros([n, 1]))
    b = tf.Variable(tf.zeros([1]))

with tf.name_scope('Model'):
    Y_ = tf.nn.softmax(tf.add(tf.matmul(X, W), b))

with tf.name_scope('Cost_function'):
    cost = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=Y_)

with tf.name_scope('Gradient_descent'):
    training_step = tf.train.AdamOptimizer(learning_rate=alpha).minimize(cost)

**Model training**

In [135]:
from sklearn.metrics import roc_auc_score

#tf.reset_default_graph()
#sess.run(tf.global_variables_initializer())

costs_train = []
aucs_train = []

costs_test = []
aucs_test = []

with tf.name_scope('Session'):
    with tf.Session() as sess:
        
        init = tf.global_variables_initializer()
        sess.run(init)
        
        for epoch in range(num_epochs):
                
                # Gradient descent
                sess.run(training_step, feed_dict={X: X_train,
                                                   Y: Y_train})
                
                # Learning curves
                costs_train.append(sess.run(cost, {X: X_train,
                                                   Y: Y_train}))
                #costs_test.append(sess.run(cost, {X: X_test,
                #                                  Y: Y_test}))
                
                # Metrics
                #train_auc = roc_auc_score(Y_train, sess.run())
                #test_auc = roc_auc_score(Y_test, )
                
                #aucs_train.append(train_auc)
                #aucs_test.append(test_auc)

## 2. Feed-forward neural networks